In [2]:
import os
import shutil
import xml.etree.ElementTree as ET
from ultralytics import YOLO
import random
import cv2
import pytesseract
import torch
import easyocr
import re
import time
import json
import re
import numpy as np
from collections import defaultdict, deque



In [ ]:
# -------------------------------
# ตรวจสอบไฟล์ภาพเสีย

train_img_dir = "images_all"

bad_files = []
for f in os.listdir(train_img_dir):
    if f.endswith((".jpg", ".png")):
        path = os.path.join(train_img_dir, f)
        img = cv2.imread(path)
        if img is None:
            bad_files.append(f)

print("จำนวนไฟล์เสีย:", len(bad_files))
print("ตัวอย่างไฟล์เสีย:", bad_files[:10])

KeyboardInterrupt: 

: 

In [ ]:
img_dir = "images_all"
lbl_dir = "labels_all" 

missing_labels = []
for f in os.listdir(img_dir):
    if f.endswith(".jpg") or f.endswith(".png"):
        name = os.path.splitext(f)[0] + ".txt"
        if not os.path.exists(os.path.join(lbl_dir, name)):
            missing_labels.append(name)

print("ไม่มี labels:", missing_labels[:10])

ไม่มี labels: []


In [ ]:


# แปลงไฟล์ .xml เป็น .txt (YOLO format)

xml_folder = "annotations_xml"       # ที่เก็บไฟล์ .xml
output_folder = "labels_after_xml"   # ที่จะเซฟไฟล์ .txt
class_mapping = {"licence": 0}       # map ชื่อ class → id (0)

# ✅ สร้างโฟลเดอร์ input ถ้าไม่มี
if not os.path.exists(xml_folder):
    os.makedirs(xml_folder)
    print(f"⚠️ โฟลเดอร์ '{xml_folder}' ถูกสร้างขึ้น (ว่างเปล่า) → กรุณาใส่ไฟล์ .xml ก่อน")

# ✅ สร้างโฟลเดอร์ output ถ้าไม่มี
os.makedirs(output_folder, exist_ok=True)

# ตรวจไฟล์ใน input
xml_files = [f for f in os.listdir(xml_folder) if f.endswith(".xml")]

if not xml_files:
    print(f"⚠️ ไม่พบไฟล์ .xml ใน '{xml_folder}'")
else:
    for xml_file in xml_files:
        tree = ET.parse(os.path.join(xml_folder, xml_file))
        root = tree.getroot()

        # ขนาดภาพ
        size = root.find("size")
        img_w = int(size.find("width").text)
        img_h = int(size.find("height").text)

        yolo_lines = []

        for obj in root.findall("object"):
            class_name = obj.find("name").text
            if class_name not in class_mapping:
                continue
            class_id = class_mapping[class_name]

            bndbox = obj.find("bndbox")
            xmin = int(bndbox.find("xmin").text)
            ymin = int(bndbox.find("ymin").text)
            xmax = int(bndbox.find("xmax").text)
            ymax = int(bndbox.find("ymax").text)

            # แปลงเป็น YOLO format
            x_center = (xmin + xmax) / 2 / img_w
            y_center = (ymin + ymax) / 2 / img_h
            w = (xmax - xmin) / img_w
            h = (ymax - ymin) / img_h

            yolo_lines.append(
                f"{class_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}"
            )

        # เซฟไฟล์ .txt
        txt_file = os.path.splitext(xml_file)[0] + ".txt"
        with open(os.path.join(output_folder, txt_file), "w") as f:
            f.write("\n".join(yolo_lines))

    print(f"✅ แปลงเสร็จแล้ว! ไฟล์ txt อยู่ที่: {output_folder}")



✅ แปลงเสร็จแล้ว! ไฟล์ txt อยู่ที่: labels_after_xml


In [ ]:


# -------------------------------
# ใช้สำหรับแบ่ง dataset เป็น train/val/test
# -------------------------------
images_dir = "images_all"   # โฟลเดอร์เก็บภาพทั้งหมด
labels_dir = "labels_all"   # โฟลเดอร์เก็บ labels ทั้งหมด

output_base = "datasets/YOLO"  # โฟลเดอร์สำหรับ train/val/test
split_ratio = (0.7, 0.2, 0.1)  # train, val, test

# -------------------------------
# สร้างโฟลเดอร์ input ถ้าไม่มี
# -------------------------------
if not os.path.exists(images_dir):
    os.makedirs(images_dir)
    print(f"⚠️ สร้างโฟลเดอร์ '{images_dir}' แล้ว (ตอนนี้ว่างเปล่า) → กรุณาใส่ไฟล์ภาพก่อน")

if not os.path.exists(labels_dir):
    os.makedirs(labels_dir)
    print(f"⚠️ สร้างโฟลเดอร์ '{labels_dir}' แล้ว (ตอนนี้ว่างเปล่า) → กรุณาใส่ไฟล์ labels ก่อน")

# -------------------------------
# สร้างโฟลเดอร์ output (images + labels + split)
# -------------------------------
for split in ["train", "val", "test"]:
    os.makedirs(os.path.join(output_base, "images", split), exist_ok=True)
    os.makedirs(os.path.join(output_base, "labels", split), exist_ok=True)

# -------------------------------
# รวมรายชื่อไฟล์ภาพ
# -------------------------------
images = [f for f in os.listdir(images_dir) if f.lower().endswith((".jpg", ".png"))]

if not images:
    print(f"⚠️ ไม่พบไฟล์ภาพใน '{images_dir}' → กรุณาใส่ภาพก่อน")
    exit()

random.shuffle(images)

n_total = len(images)
n_train = int(split_ratio[0] * n_total)
n_val = int(split_ratio[1] * n_total)
n_test = n_total - n_train - n_val

print(f"พบทั้งหมด {n_total} รูป → Train={n_train}, Val={n_val}, Test={n_test}")

# -------------------------------
# ฟังก์ชันย้ายไฟล์
# -------------------------------
def move_files(file_list, split):
    for img_file in file_list:
        src_img = os.path.join(images_dir, img_file)
        dst_img = os.path.join(output_base, "images", split, img_file)

        # path ของ label (ชื่อเดียวกันแต่ .txt)
        label_file = os.path.splitext(img_file)[0] + ".txt"
        src_lbl = os.path.join(labels_dir, label_file)
        dst_lbl = os.path.join(output_base, "labels", split, label_file)

        # copy ภาพ
        shutil.copy(src_img, dst_img)

        # copy label ถ้ามี
        if os.path.exists(src_lbl):
            shutil.copy(src_lbl, dst_lbl)
        else:
            print(f"⚠️ ไม่มี label สำหรับ {img_file}")

# -------------------------------
# แบ่ง dataset
# -------------------------------
move_files(images[:n_train], "train")
move_files(images[n_train:n_train+n_val], "val")
move_files(images[n_train+n_val:], "test")

print("✅ แบ่ง dataset เสร็จแล้ว → อยู่ใน:", output_base)


พบทั้งหมด 1402 รูป → Train=981, Val=280, Test=141
✅ แบ่ง dataset เสร็จแล้ว → อยู่ใน: datasets/YOLO


In [1]:


print(torch.__version__)
print(torch.cuda.is_available())    # 11.8
print(torch.cuda.get_device_name(0))  # แสดงชื่อ GPU
print(torch.cuda.memory_allocated(0))
print(torch.cuda.memory_reserved(0))

NameError: name 'torch' is not defined

In [ ]:
images_dir = "images_all"  
torch.cuda.empty_cache()
data_yaml = """
train: D:/yolo/project_2/datasets/YOLO/images/train
val:   D:/yolo/project_2/datasets/YOLO/images/val
test:  D:/yolo/project_2/datasets/YOLO/images/test
nc: 1
names: ['license_plate']
"""
with open("datasets/YOLO/data.yaml", "w") as f:
    f.write(data_yaml)

data_yaml  = "datasets/YOLO/data.yaml"
# -------------------------------


# เลือกโมเดล
model = YOLO("yolov8s.pt")  # small model, VRAM 8GB ยังรองรับ

# Train
model.train(
    data=data_yaml,
    epochs=100,
    imgsz=560,       # ขนาดภาพใหญ่ขึ้น
    batch=8,        # เพิ่ม batch size ให้ใช้ RAM 32GB
    name="plate_detector",
    augment=True,    # เพิ่มความแม่น
    device=0 ,         # ใช้ GPU
    half=True     # FP16 ลด VRAM
    
)

# ทดสอบโมเดล
results = model.predict(
    source=images_dir,
    save=True,
    conf=0.5
)
print("✅ Prediction finished, check runs/detect/plate_detector/")



Ultralytics 8.3.201  Python-3.10.18 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1060, 6144MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=datasets/YOLO/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=True, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=560, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=plate_detector, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=

In [3]:
model2 = YOLO("runs/detect/plate_detector/weights/best.pt")

In [6]:
results = model2.predict(
    source="car02.jpg",  # folder
    conf=0.5,
    save=True
)
# ดูผลลัพธ์เป็น array
print(results[0].boxes.xyxy)   # [x1, y1, x2, y2] ของ bounding boxes
print(results[0].boxes.conf)   # confidence
print(results[0].boxes.cls)    # class index
results[0].show()


image 1/1 d:\yolo\Project_2\car02.jpg: 448x576 1 license_plate, 16.9ms
Speed: 1.9ms preprocess, 16.9ms inference, 1.7ms postprocess per image at shape (1, 3, 448, 576)
Results saved to D:\yolo\Project_2\runs\detect\predict6
tensor([[524.0804, 766.9173, 795.0178, 866.7012]], device='cuda:0')
tensor([0.8375], device='cuda:0')
tensor([0.], device='cuda:0')


In [ ]:
# ============================
# 2) ตั้งค่า pytesseract
# ============================
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# ============================
# 3) เปิดกล้อง USB
# ============================
cap = cv2.VideoCapture(0)   # 0 = default camera, ถ้ามีหลายตัวเปลี่ยนเป็น 1,2,...

if not cap.isOpened():
    print("❌ ไม่สามารถเปิดกล้องได้")
    exit()

# ============================
# 4) Loop รับภาพจากกล้อง
# ============================
while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ อ่านภาพจากกล้องไม่ได้")
        break

    # ----------------------------
    # YOLO ตรวจจับป้ายทะเบียน
    # ----------------------------
    results = model2(frame, conf=0.5)

    for r in results:
        boxes = r.boxes.xyxy.cpu().numpy()  # [x1,y1,x2,y2,conf,class]
        clss = r.boxes.cls.cpu().numpy()

        for box, cls in zip(boxes, clss):
            x1, y1, x2, y2 = map(int, box[:4])
            label = int(cls)

            # วาดกรอบรอบป้ายทะเบียน
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # ตัดเฉพาะป้ายทะเบียน
            crop_plate = frame[y1:y2, x1:x2]

            if crop_plate.size > 0:
                # OCR อ่านตัวอักษร
                text = pytesseract.image_to_string(crop_plate, lang="tha", config="--psm 7")
                text = text.strip()

                # แสดงผล OCR เหนือกรอบ
                cv2.putText(frame, text, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # แสดงภาพบนจอ
    cv2.imshow("License Plate Detection", frame)

    # กด q เพื่อออก
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


0: 448x576 (no detections), 18.0ms
Speed: 11.7ms preprocess, 18.0ms inference, 2.4ms postprocess per image at shape (1, 3, 448, 576)

0: 448x576 (no detections), 17.1ms
Speed: 3.7ms preprocess, 17.1ms inference, 1.5ms postprocess per image at shape (1, 3, 448, 576)

0: 448x576 (no detections), 18.3ms
Speed: 1.9ms preprocess, 18.3ms inference, 0.8ms postprocess per image at shape (1, 3, 448, 576)

0: 448x576 (no detections), 17.8ms
Speed: 3.0ms preprocess, 17.8ms inference, 1.5ms postprocess per image at shape (1, 3, 448, 576)

0: 448x576 (no detections), 18.5ms
Speed: 2.8ms preprocess, 18.5ms inference, 0.8ms postprocess per image at shape (1, 3, 448, 576)

0: 448x576 (no detections), 18.5ms
Speed: 1.9ms preprocess, 18.5ms inference, 0.8ms postprocess per image at shape (1, 3, 448, 576)

0: 448x576 (no detections), 17.1ms
Speed: 3.1ms preprocess, 17.1ms inference, 1.3ms postprocess per image at shape (1, 3, 448, 576)

0: 448x576 (no detections), 18.6ms
Speed: 1.6ms preprocess, 18.6ms 

KeyboardInterrupt: 

In [7]:
import cv2
import torch
from ultralytics import YOLO
import easyocr

# --------------------------
# กำหนดอุปกรณ์ (GPU / CPU)
# --------------------------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

# --------------------------
# โหลด YOLOv8 (ใช้ FP16 บน GPU)
# --------------------------

model2.fuse()

# --------------------------
# OCR Engine (EasyOCR)
# --------------------------
reader = easyocr.Reader(['en', 'th'], gpu=(DEVICE == "cuda"))

# --------------------------
# ฟังก์ชัน preprocess
# --------------------------
def preprocess_plate(img):
    img = cv2.resize(img, (320, 96), interpolation=cv2.INTER_CUBIC)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    thresh = cv2.adaptiveThreshold(
        gray, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY, 21, 9
    )
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced = clahe.apply(morph)
    return enhanced

# --------------------------
# Main Loop รับภาพจากกล้อง
# --------------------------
def run_camera(cam_index=0, conf_thres=0.5):
    cap = cv2.VideoCapture(cam_index)

    if not cap.isOpened():
        print("[ERROR] ไม่สามารถเปิดกล้องได้")
        return

    print("[INFO] เริ่มระบบ ALPR (กด q เพื่อออก)")
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # YOLO Detect
        results = model2.predict(
            source=frame,
            conf=conf_thres,
            imgsz=640,
            device=DEVICE,
            half=True,
            verbose=False
        )

        # Loop ทุก detection
        for r in results:
            for box in r.boxes.xyxy.cpu().numpy():
                x1, y1, x2, y2 = map(int, box[:4])
                crop = frame[y1:y2, x1:x2]

                if crop.size == 0:
                    continue

                # Preprocess
                processed = preprocess_plate(crop)

                # OCR
                ocr_result = reader.readtext(processed, detail=0)

                plate_text = ocr_result[0] if ocr_result else "?"
                print("Detected:", plate_text)

                # วาด Bounding Box + Text
                cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 2)
                cv2.putText(frame, plate_text, (x1, y1-10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)

        cv2.imshow("ALPR Camera", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# --------------------------
# Run
# --------------------------
if __name__ == "__main__":
    run_camera(cam_index=0)


Using device: cuda
[INFO] เริ่มระบบ ALPR (กด q เพื่อออก)
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: -
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: -= --
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Detected: ?
Det

In [6]:
"""
camera_plate_pipeline.py

Pipeline: YOLO detection -> crop plate -> preprocess -> OCR (Tesseract only)
-> regex + dictionary validation -> multi-frame voting -> save event

ต้องมีไฟล์:
 - valid_prefixes.txt
 - provinces.txt
ในโฟลเดอร์เดียวกับสคริปต์
"""

# -------------------- CONFIG --------------------
TESSERACT_CMD = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
CAM_INDEX = 0
CONF_THRESHOLD = 0.35
VOTE_WINDOW = 7
MIN_PLATE_HEIGHT_PX = 30
OUTPUT_EVENTS_DIR = "events"
DEVICE = 0 if torch.cuda.is_available() else 'cpu'
# ------------------------------------------------

# ตั้งค่า pytesseract
pytesseract.pytesseract.tesseract_cmd = TESSERACT_CMD

# โหลด dictionary
def load_list(filepath):
    if not os.path.exists(filepath):
        print(f"[WARN] ไม่พบไฟล์ {filepath} — สร้างไฟล์และเพิ่มข้อมูลก่อนรัน")
        return set()
    with open(filepath, "r", encoding="utf-8") as f:
        return set(line.strip() for line in f if line.strip())

VALID_PREFIXES = load_list("valid_prefixes.txt")
VALID_PROVINCES = load_list("provinces.txt")

# regex pattern
PLATE_PATTERN = re.compile(r"([ก-ฮ]{1,2})\s?(\d{1,4})(?:\s?([ก-๙]+))?")

# สร้างโฟลเดอร์ตอนบันทึก
os.makedirs(OUTPUT_EVENTS_DIR, exist_ok=True)

# -------------------- Utility / Preprocess --------------------
def preprocess_plate(plate_img):
    h, w = plate_img.shape[:2]
    if h < MIN_PLATE_HEIGHT_PX:
        scale = MIN_PLATE_HEIGHT_PX / max(1, h)
        plate_img = cv2.resize(plate_img, None, fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)
    gray = cv2.cvtColor(plate_img, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    gray = clahe.apply(gray)
    gray = cv2.fastNlMeansDenoising(gray, None, 10, 7, 21)
    th = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                               cv2.THRESH_BINARY, 31, 2)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,2))
    th = cv2.morphologyEx(th, cv2.MORPH_CLOSE, kernel)
    return th

def ocr_tesseract(image):
    config = "--oem 3 --psm 7 -l tha+eng"
    data = pytesseract.image_to_data(image, config=config, output_type=pytesseract.Output.DICT)
    texts, confs = [], []
    for txt, conf in zip(data.get('text', []), data.get('conf', [])):
        if txt.strip()=="": continue
        try: c=float(conf)
        except: c=0.0
        texts.append(txt.strip())
        confs.append(max(0.0,c))
    text = " ".join(texts).strip()
    avg_conf = float(np.mean(confs)) if confs else 0.0
    return text, avg_conf

def clean_plate_text(text):
    if not text or text.strip()=="": return None
    text = re.sub(r"[^ก-ฮ0-9\s]", "", text).strip()
    m = PLATE_PATTERN.search(text)
    if not m: return None
    prefix, number, province = m.groups()
    if prefix not in VALID_PREFIXES: return None
    if province:
        if province not in VALID_PROVINCES:
            return f"{prefix} {number}"
        else:
            return f"{prefix} {number} {province}"
    return f"{prefix} {number}"

# -------------------- Voting & Keying --------------------
def make_key_from_bbox(bbox, bucket=80):
    x1,y1,x2,y2 = bbox
    cx = int((x1+x2)/2)
    cy = int((y1+y2)/2)
    return (cx//bucket, cy//bucket)

votes = defaultdict(lambda: deque(maxlen=VOTE_WINDOW))
last_saved_time = defaultdict(lambda: 0.0)

# -------------------- Load YOLO Model --------------------
model2.fuse()
model2.to(DEVICE)
model2.model.half()

# -------------------- Main Loop --------------------
def run_camera():
    cap = cv2.VideoCapture(CAM_INDEX)
    if not cap.isOpened():
        print("[ERROR] ไม่สามารถเปิดกล้องได้. ตรวจสอบ CAM_INDEX.")
        return

    print("[INFO] เริ่ม pipeline. กด 'q' เพื่อออก.")
    while True:
        ret, frame = cap.read()
        if not ret:
            print("[WARN] อ่าน frame ไม่ได้ ตัดการทำงาน")
            break

        results = model2(frame, device=DEVICE, conf=CONF_THRESHOLD, verbose=False)

        for r in results:
            for box, conf, cls in zip(r.boxes.xyxy, r.boxes.conf, r.boxes.cls):
                x1,y1,x2,y2 = map(int, box.cpu().numpy())
                pad = 5
                x1p = max(0, x1-pad); y1p = max(0, y1-pad)
                x2p = min(frame.shape[1], x2+pad); y2p = min(frame.shape[0], y2+pad)
                crop = frame[y1p:y2p, x1p:x2p]
                if crop.size==0: continue

                processed = preprocess_plate(crop)

                # OCR (เฉพาะ Tesseract)
                raw_text, raw_conf = ocr_tesseract(processed)

                plate_norm = clean_plate_text(raw_text)

                key = make_key_from_bbox((x1,y1,x2,y2))
                votes[key].append((plate_norm, raw_conf, time.time()))

                # decide consensus
                vote_texts=[v[0] for v in votes[key] if v[0]]
                if vote_texts:
                    candidate=max(set(vote_texts), key=vote_texts.count)
                    confs=[v[1] for v in votes[key] if v[0]==candidate]
                    avg_conf=float(np.mean(confs)) if confs else 0.0

                    if vote_texts.count(candidate)>=max(2,VOTE_WINDOW//2) and avg_conf>=40:
                        now=time.time()
                        if now-last_saved_time[key]>10:
                            last_saved_time[key]=now
                            ts=time.strftime("%Y%m%d_%H%M%S")
                            fname_base=f"{ts}_{key[0]}_{key[1]}"
                            fullpath=os.path.join(OUTPUT_EVENTS_DIR,fname_base+".jpg")
                            crop_path=os.path.join(OUTPUT_EVENTS_DIR,fname_base+"_plate.jpg")
                            json_path=os.path.join(OUTPUT_EVENTS_DIR,fname_base+".json")
                            cv2.imwrite(fullpath, frame)
                            cv2.imwrite(crop_path, crop)
                            meta={
                                "plate": candidate,
                                "avg_conf": avg_conf,
                                "timestamp": ts,
                                "bbox": [int(x1),int(y1),int(x2),int(y2)]
                            }
                            with open(json_path,"w",encoding="utf-8") as jf:
                                json.dump(meta,jf,ensure_ascii=False, indent=2)
                            print(f"[SAVED] {candidate} -> {json_path}")

                        cv2.rectangle(frame,(x1p,y1p),(x2p,y2p),(0,255,0),2)
                        cv2.putText(frame,f"{candidate} {avg_conf:.0f}",(x1p,y1p-10),
                                    cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,255,0),2)
                    else:
                        if plate_norm:
                            cv2.rectangle(frame,(x1p,y1p),(x2p,y2p),(0,165,255),2)
                            cv2.putText(frame,f"maybe:{plate_norm}",(x1p,y1p-10),
                                        cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,165,255),1)
                else:
                    cv2.rectangle(frame,(x1p,y1p),(x2p,y2p),(255,0,0),1)

                # --- แสดงผล OCR ---
                if raw_text:
                    cv2.putText(frame, f"T:{raw_text} {raw_conf:.0f}",
                                (x1p, y2p+20), cv2.FONT_HERSHEY_SIMPLEX,
                                0.6, (0,255,255),1)

        cv2.imshow("ALPR Pipeline", frame)
        if cv2.waitKey(1) & 0xFF==ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__=="__main__":
    run_camera()


[INFO] เริ่ม pipeline. กด 'q' เพื่อออก.
[SAVED] ง 9999 -> events\20250918_152828_4_2.json
[SAVED] ง 9999 -> events\20250918_152839_3_2.json
[SAVED] ง 7999 -> events\20250918_152850_3_1.json
[SAVED] ง 9999 -> events\20250918_152903_4_2.json
